In [75]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [76]:
cur.execute("""SELECT *
                FROM Unique_Teams
                ;""")
unique_team_df = pd.DataFrame(cur.fetchall())
unique_team_df.columns = [x[0] for x in cur.description]
unique_team_df.set_index('Unique_Team_ID', inplace = True)
unique_team_df.head()

,TeamName
Unique_Team_ID,
1,Bayern Munich
2,Dortmund
3,Leverkusen
4,RB Leipzig
5,Schalke 04


In [77]:
cur.execute("""SELECT HomeTeam, SUM(FTHG) as Total_Home_Goals
                FROM Matches
                WHERE Season = 2011 and Div != 'E0'
                GROUP BY HomeTeam
                 ;""")
team_goals_home_df = pd.DataFrame(cur.fetchall())
team_goals_home_df.columns = [x[0] for x in cur.description]
team_goals_home_df.head()

,HomeTeam,Total_Home_Goals
0,Aachen,15
1,Augsburg,20
2,Bayern Munich,49
3,Bochum,23
4,Braunschweig,21


In [78]:
cur.execute("""SELECT AwayTeam, SUM(FTAG) as Total_Away_Goals
                FROM Matches
                WHERE Season = 2011 and Div != 'E0'
                GROUP BY AwayTeam
                 ;""")
team_goals_away_df = pd.DataFrame(cur.fetchall())
team_goals_away_df.columns = [x[0] for x in cur.description]
team_goals_away_df.head()

,AwayTeam,Total_Away_Goals
0,Aachen,15
1,Augsburg,16
2,Bayern Munich,28
3,Bochum,18
4,Braunschweig,16


In [79]:
Home_Away_Goals = pd.concat([team_goals_away_df, team_goals_home_df], axis = 1)
Home_Away_Goals.drop(['HomeTeam'], axis = 1, inplace =True)
Home_Away_Goals.rename(columns = {'AwayTeam': 'Team_Name'}, inplace = True)
Home_Away_Goals.head()

,Team_Name,Total_Away_Goals,Total_Home_Goals
0,Aachen,15,15
1,Augsburg,16,20
2,Bayern Munich,28,49
3,Bochum,18,23
4,Braunschweig,16,21


In [80]:
Home_Away_Goals['Total_Goals'] = Home_Away_Goals['Total_Away_Goals'] + Home_Away_Goals['Total_Home_Goals']
Home_Away_Goals.head()

,Team_Name,Total_Away_Goals,Total_Home_Goals,Total_Goals
0,Aachen,15,15,30
1,Augsburg,16,20,36
2,Bayern Munich,28,49,77
3,Bochum,18,23,41
4,Braunschweig,16,21,37


In [81]:
cur.execute("""SELECT HomeTeam, COUNT(FTR) as Home_Win
                FROM Matches
                WHERE Season = 2011 and FTR = 'H' and Div != 'E0'
                GROUP BY HomeTeam
                ;""")
home_wins_df = pd.DataFrame(cur.fetchall())
home_wins_df.columns = [x[0] for x in cur.description]
home_wins_df.head()

,HomeTeam,Home_Win
0,Aachen,4
1,Augsburg,6
2,Bayern Munich,14
3,Bochum,7
4,Braunschweig,6


In [82]:
cur.execute("""SELECT AwayTeam, COUNT(FTR) as Away_Win
                FROM Matches
                WHERE Season = 2011 and FTR = 'A'and Div != 'E0'
                GROUP BY AwayTeam
                ;""")
away_wins_df = pd.DataFrame(cur.fetchall())
away_wins_df.columns = [x[0] for x in cur.description]
away_wins_df.head()

,AwayTeam,Away_Win
0,Aachen,2
1,Augsburg,2
2,Bayern Munich,9
3,Bochum,3
4,Braunschweig,4


In [83]:
total_wins = pd.concat([away_wins_df, home_wins_df], axis = 1)
total_wins.drop(['HomeTeam'], axis = 1, inplace =True)
total_wins.rename(columns = {'AwayTeam': 'Team_Name'}, inplace = True)
total_wins.head()

,Team_Name,Away_Win,Home_Win
0,Aachen,2,4
1,Augsburg,2,6
2,Bayern Munich,9,14
3,Bochum,3,7
4,Braunschweig,4,6


In [84]:
total_wins['Total_Wins'] = total_wins['Away_Win'] + total_wins['Home_Win']
total_wins.head()

,Team_Name,Away_Win,Home_Win,Total_Wins
0,Aachen,2,4,6
1,Augsburg,2,6,8
2,Bayern Munich,9,14,23
3,Bochum,3,7,10
4,Braunschweig,4,6,10


In [85]:
cur.execute("""SELECT distinct HomeTeam,   
                CASE
                WHEN FTR = 'A' THEN SUM(FTAG)
                WHEN FTR = 'H' THEN SUM(FTHG)
                WHEN FTR = 'D' THEN SUM(FTAG)
                ELSE 0 END AS "Total goals"
                ,SUM(CASE WHEN FTR = 'H' then 1 else 0 end) "Home_Wins"
                ,SUM(CASE WHEN FTR = 'A' then 1 ELSE 0 END) "Home_Losses"
                FROM matches
                WHERE season = 2011 and Div != 'E0'
                GROUP BY 1 
                ORDER BY 1
                ;""")
total_wins_loss = pd.DataFrame(cur.fetchall())
total_wins_loss.columns = [x[0] for x in cur.description]
total_wins_loss.head()

,HomeTeam,Total goals,Home_Wins,Home_Losses
0,Aachen,24,4,7
1,Augsburg,19,6,4
2,Bayern Munich,6,14,2
3,Bochum,23,7,7
4,Braunschweig,15,6,3


In [86]:
cur.execute("""SELECT distinct AwayTeam,   
                CASE
                WHEN FTR = 'A' THEN SUM(FTAG)
                WHEN FTR = 'H' THEN SUM(FTHG)
                WHEN FTR = 'D' THEN SUM(FTAG)
                ELSE 0 END AS "Total goals"
                ,SUM(CASE WHEN FTR = 'A' then 1 else 0 end) "Away_Wins"
                ,SUM(CASE WHEN FTR = 'H' then 1 ELSE 0 END) "Away_Losses"
                FROM matches
                WHERE season = 2011 and Div != 'E0'
                GROUP BY 1 
                ORDER BY 1
                ;""")
total_wins_loss_away = pd.DataFrame(cur.fetchall())
total_wins_loss_away.columns = [x[0] for x in cur.description]
total_wins_loss_away.head()

,AwayTeam,Total goals,Away_Wins,Away_Losses
0,Aachen,23,2,8
1,Augsburg,30,2,8
2,Bayern Munich,28,9,5
3,Bochum,32,3,10
4,Braunschweig,20,4,6


In [97]:
big_df = pd.concat([total_wins_loss, total_wins_loss_away, Home_Away_Goals], axis = 1)
big_df.drop(['AwayTeam', 'Total goals', 'Total_Away_Goals', 'Total_Home_Goals', 'HomeTeam'], axis = 1, inplace =True)
big_df.head()

,Home_Wins,Home_Losses,Away_Wins,Away_Losses,Team_Name,Total_Goals
0,4,7,2,8,Aachen,30
1,6,4,2,8,Augsburg,36
2,14,2,9,5,Bayern Munich,77
3,7,7,3,10,Bochum,41
4,6,3,4,6,Braunschweig,37


In [98]:
big_df['Total_Wins'] = big_df['Away_Wins'] + big_df['Home_Wins']
big_df['Total_Losses'] = big_df['Home_Losses'] + big_df['Away_Losses']
big_df.head()

,Home_Wins,Home_Losses,Away_Wins,Away_Losses,Team_Name,Total_Goals,Total_Wins,Total_Losses
0,4,7,2,8,Aachen,30,6,15
1,6,4,2,8,Augsburg,36,8,12
2,14,2,9,5,Bayern Munich,77,23,7
3,7,7,3,10,Bochum,41,10,17
4,6,3,4,6,Braunschweig,37,10,9


In [99]:
big_df.drop(['Home_Wins', 'Home_Losses', 'Away_Wins', 'Away_Losses'], axis = 1, inplace = True)
big_df.head()

,Team_Name,Total_Goals,Total_Wins,Total_Losses
0,Aachen,30,6,15
1,Augsburg,36,8,12
2,Bayern Munich,77,23,7
3,Bochum,41,10,17
4,Braunschweig,37,10,9


MAKE HISTOGRAM FOR EACH GERMAN TEAM WINS AND LOSSES

USE DARKSKY API TO FIND BERLIN WEATHER FOR MATCHES 